In [ ]:
#default_exp protonets
#export
import warnings

import wandb

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

from unsupervised_meta_learning.nn_utils import Flatten, get_proto_accuracy, conv3x3
from unsupervised_meta_learning.pl_dataloaders import OmniglotDataModule

In [ ]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [ ]:
#export
class PrototypicalNetwork(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_size=64):
        super(PrototypicalNetwork, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.hidden_size = hidden_size

        self.encoder = nn.Sequential(
            conv3x3(in_channels, hidden_size),
            conv3x3(hidden_size, hidden_size),
            conv3x3(hidden_size, hidden_size),
            conv3x3(hidden_size, out_channels)
        )
    
    def forward(self, inputs):
        embeddings = self.encoder(inputs.view(-1, *inputs.shape[2:]))
        return embeddings.view(*inputs.shape[:2], -1)

In [ ]:
#export
class ProtoModule(pl.LightningModule):
    def __init__(self, encoder, num_classes, **kwargs):
        super().__init__()
        self.model = encoder
        self.automatic_optimization = True
        self.num_classes_per_task = num_classes
        self.acccuracy = get_proto_accuracy
    
    def forward(self, x):
        return self.model(x)
    
    def get_prototypes(self, emb, targets, num_classes):
        """Compute the prototypes (the mean vector of the embedded training/support 
        points belonging to its class) for each classes in the task.
        Parameters
        ----------
        embeddings : `torch.FloatTensor` instance
            A tensor containing the embeddings of the support points. This tensor 
            has shape `(batch_size, num_examples, embedding_size)`.
        targets : `torch.LongTensor` instance
            A tensor containing the targets of the support points. This tensor has 
            shape `(batch_size, num_examples)`.
        num_classes : int
            Number of classes in the task.
        Returns
        -------
        prototypes : `torch.FloatTensor` instance
            A tensor containing the prototypes for each class. This tensor has shape
            `(batch_size, num_classes, embedding_size)`.
        """

        batch_size, emb_size = emb.size(0), emb.size(-1)

        num_samples = self.get_num_samples(targets, num_classes, dtype=emb.dtype)
        num_samples.unsqueeze_(-1)
        num_samples = torch.max(num_samples, torch.ones_like(num_samples))

        prototypes = emb.new_zeros((batch_size, num_classes, emb_size))
        indices = targets.unsqueeze(-1).expand_as(emb)

        prototypes.scatter_add_(1, indices, emb).div_(num_samples)

        return prototypes
    
    def get_num_samples(self, targets, num_classes, dtype=None):
        batch_size = targets.size(0)
        with torch.no_grad():
            ones = torch.ones_like(targets, dtype=dtype)
            num_samples = ones.new_zeros((batch_size, num_classes))
            num_samples.scatter_add_(1, targets, ones)
        return num_samples
    
    def protoypical_loss(self, prototypes, emb, targets, **kwargs):
        """Compute the loss (i.e. negative log-likelihood) for the prototypical 
        network, on the test/query points.
        Parameters
        ----------
        prototypes : `torch.FloatTensor` instance
            A tensor containing the prototypes for each class. This tensor has shape 
            `(batch_size, num_classes, embedding_size)`.
        embeddings : `torch.FloatTensor` instance
            A tensor containing the embeddings of the query points. This tensor has 
            shape `(batch_size, num_examples, embedding_size)`.
        targets : `torch.LongTensor` instance
            A tensor containing the targets of the query points. This tensor has 
            shape `(batch_size, num_examples)`.
        Returns
        -------
        loss : `torch.FloatTensor` instance
            The negative log-likelihood on the query points.
        """
        squared_distances = torch.sum(
            (prototypes.unsqueeze(2) - emb.unsqueeze(1)) ** 2, dim = -1
        )
        return F.cross_entropy(-squared_distances, targets, **kwargs)
        
    
    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optim
    
    def training_step(self, batch, batch_idx):
        train_inputs, train_targets = batch['train']
        train_inputs, train_targets = train_inputs.to(self.device), train_targets.to(self.device)
        test_inputs, test_targets = batch['test']
        test_inputs, test_targets = test_inputs.to(self.device), test_targets.to(self.device)
        
        optimizer = self.optimizers()
        
        self.model.zero_grad()
        
        train_emb = self.model(train_inputs)
        test_emb = self.model(test_inputs)
        
        prototypes = self.get_prototypes(train_emb, train_targets, self.num_classes_per_task)
        loss = self.protoypical_loss(prototypes, train_emb, train_targets)
        
        with torch.no_grad():
            acc = get_proto_accuracy(prototypes, test_emb, test_targets)
        
#         loss.backward()
#         optimizer.zero_grad()
#         self.manual_backward(loss, optimizer)
#         optimizer.step()
        
        self.log_dict({
            "loss": loss.item(),
            "accuracy": acc.item()
        }, prog_bar=True)
        return loss
        
        

In [ ]:
model = ProtoModule(encoder=PrototypicalNetwork(1, 64, hidden_size=64), num_classes=5)

In [ ]:
dm = OmniglotDataModule(
        "data",
        shots=5,
        ways=5,
        shuffle_ds=True,
        test_shots=15,
        meta_train=True,
        download=True,
        batch_size=16,
        shuffle=True,
        num_workers=8,
)

In [ ]:
logger = WandbLogger(
    project='protonet',
    config={
        'batch_size': 16,
        'steps': 100,
        'dataset': "omniglot"
    }
)
trainer = pl.Trainer(
        profiler='simple',
        max_epochs=100,
        max_steps=100,
        fast_dev_run=False,
        num_sanity_val_steps=2, gpus=1, logger=logger
    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    trainer.fit(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: p0int (use `wandb login --relogin` to force relogin)



  | Name  | Type                | Params
----------------------------------------------
0 | model | PrototypicalNetwork | 111 K 
----------------------------------------------
111 K     Trainable params
0         Non-trainable params
111 K     Total params
0.448     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  26.957         	|  100 %          	|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  8.9848         	|1              	|  8.9848         	|  33.33          	|
run_training_batch                 	|  0.041832       	|100            	|  4.1832         	|  15.518         	|
get_train_batch                    	|  0.039948       	|100            	|  3.9948         	|  14.819         	|
optimizer_step_and_closure_0       	|  0.033977       	|100            	|  3.3977         	|  12.604         	|
training_step_and_backward         

In [ ]:
wandb.finish()

loss,0.00538
accuracy,0.95917
epoch,0
trainer/global_step,99
_runtime,12
_timestamp,1622904145
_step,1


loss,█▁
accuracy,▁█
epoch,▁▁
trainer/global_step,▁█
_runtime,▁█
_timestamp,▁█
_step,▁█


In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 01_nn_utils.ipynb.
Converted 01b_data_loaders_pl.ipynb.
Converted 02_maml_pl.ipynb.
Converted 03_protonet_pl.ipynb.
Converted 04_cactus.ipynb.
Converted index.ipynb.
